In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from numpy.linalg import inv
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession\
  .builder\
  .appName('scratch-ALS')\
  .getOrCreate()
sc = spark.sparkContext

In [ ]:
numWorkers = sc.defaultParallelism
numWorkers

2

In [ ]:
def getRelativeIndex(value, index_list):
  return index_list[value]

In [ ]:
def sortByRelativeIndex(user_or_item, input):
  if user_or_item == 'user':
    return input\
      .map(lambda x: x[1])\
      .distinct()\
      .sortBy(lambda x: x, ascending=True)\
      .zipWithIndex().collect()
  else: return input\
      .map(lambda x: x[2][1])\
      .distinct()\
      .sortBy(lambda x: x, ascending=True)\
      .zipWithIndex().collect()


In [ ]:
def getBlock(user_or_item, ratings, sorted_users, sorted_items):
  if user_or_item == 'user':
    return ratings\
      .map(lambda x: (getRelativeIndex(x[0], sorted_users), getRelativeIndex(x[1][0], sorted_items)))\
      .groupByKey()
  else:
    return ratings\
      .map(lambda x: (getRelativeIndex(x[1][0], sorted_items), getRelativeIndex(x[0], sorted_users)))\
      .groupByKey()


In [ ]:
data = sc.textFile("u.data")
raw_data = data.map(
  lambda l: l.split('\t')
).map(lambda l: (int(l[0]), int(l[1]), float(l[2])))

In [ ]:
sorted_users = dict(raw_data.map(lambda x: x[0]).distinct().sortBy(lambda idx: idx, ascending = True)\
  .zipWithIndex().collect())

sorted_items = dict(raw_data.map(lambda x: x[1]).distinct().sortBy(lambda idx: idx, ascending = True)\
  .zipWithIndex().collect())

item_count = len(sorted_items)
user_count = len(sorted_users)

In [ ]:
M = raw_data.map(lambda l: (l[0], (l[1], l[2])))\
.map(lambda x: (getRelativeIndex(x[0], sorted_users), (getRelativeIndex(x[1][0], sorted_items), x[1][1])))

In [ ]:
numFactors = 10
W = np.matrix(np.random.rand(numFactors, user_count))
H = np.matrix(np.random.rand(numFactors, item_count))

In [ ]:
R_u = M.map(lambda x: (x[0], (x[1][0], x[1][1]))).cache()
R_i = M.map(lambda x: (x[1][0], (x[0], x[1][1]))).cache()

In [ ]:
w_broadcast = sc.broadcast(W)
h_broadcast = sc.broadcast(H)

In [ ]:
def computeOptimizeMatrix(iterables, constant_matrix_broadcast, lamb):
  fixed_matrix = constant_matrix_broadcast.value
  num_factors = fixed_matrix.shape[0]
  r_i = np.zeros((1, num_factors))
  iter_dict = dict(iterables)
  XtX = np.zeros((num_factors, num_factors))
  for i, val in enumerate(r_i[0]):
    if i in iter_dict:
      r_i[0, i] = iter_dict[i]
  updated_vector = (inv((fixed_matrix).dot(fixed_matrix.T) + lamb.value * np.eye(num_factors))).dot((fixed_matrix.dot(r_i.T)))
  return updated_vector

In [ ]:
def computeOptimizeMatrix(iterables, constant_matrix_broadcast, lamb):
  fixed_matrix = constant_matrix_broadcast.value
  num_factors = fixed_matrix.shape[0]
  r_i = np.zeros((1, num_factors))
  iter_dict = dict(iterables)
  # XtX = np.zeros((num_factors, num_factors))
  # XXt = fixed_matrix.dot(fixed_matrix.T)
  # XtX = np.zeros((numFactors, numFactors))
  # RX = np.zeros((numFactors, 1))
  # for i in iter_dict.items():
  #   index = i[0] - 1
  #   rating = i[1]
  #   C = H[:, [index]]
  #   RX += (rating) * C
  #   XtX += (C.dot(C.T))
  X = fixed_matrix[:, list(iter_dict.keys())]
  R = np.matrix(list(iter_dict.values()))
  XtX = X.dot(X.T)
  RX = X.dot(R.T)
  return np.linalg.solve(XtX + lamb.value * np.eye(num_factors),RX)

In [ ]:
# random_user = R_u.groupByKey().collect()[6]
# iterables = list(random_user[1])
# constant_matrix_broadcast = h_broadcast

# fixed_matrix = constant_matrix_broadcast.value
# num_factors = fixed_matrix.shape[0]

# iter_dict = dict(iterables)
# XtX = np.zeros((num_factors, num_factors))
# XXt = fixed_matrix.dot(fixed_matrix.T)
# XtX = np.zeros((numFactors, numFactors))
# RX = np.zeros((numFactors, 1))
# for i in iter_dict.items():
#   index = sorted_items[i[0]]
#   rating = i[1]
#   C = H[:, [index]]
#   RX += rating * C
#   XtX += (C.dot(C.T))
# np.linalg.solve(XtX, RX)

In [ ]:
LAMBDA = 0.01   # regularization
np.random.seed(42)


def get_rmse(R, ms: np.ndarray, us: np.ndarray) -> np.float64:
    diff = R - ms * us.T
    return np.sqrt(np.sum(np.power(diff, 2)) / (M_count * U_count))


def update(i: int, mat: np.ndarray, ratings: np.ndarray) -> np.ndarray:
    uu = mat.shape[0]
    ff = mat.shape[1]

    XtX = mat.T * mat
    Xty = mat.T * ratings[i, :].T

    for j in range(ff):
        XtX[j, j] += LAMBDA * uu

    return np.linalg.solve(XtX, Xty)


In [ ]:
lamb = sc.broadcast(0.01)

In [ ]:
newW = R_u.groupByKey()\
  .mapValues(lambda row:computeOptimizeMatrix(row,h_broadcast,lamb))\
  .sortByKey()\
  .map(lambda data: data[1])\
  .collect()


In [ ]:
print(W.shape)
print(H.shape)

(10, 943)
(10, 1682)


In [ ]:
R_i.max(lambda x: x[1][0])

(57, (942, 4.0))

In [ ]:
W[0].shape

(1, 943)

In [ ]:
def get_error_square(rating, i, j):
  pred = W[:, [i]].T.dot(H[:, [j]])
  return (rating - pred)**2

In [ ]:
R_i.groupByKey()

PythonRDD[44] at RDD at PythonRDD.scala:53

In [ ]:
ITERATIONS = 50
for i in range(ITERATIONS):
  newW = R_u.groupByKey()\
    .mapValues(lambda row:computeOptimizeMatrix(row,h_broadcast,lamb))\
    .sortByKey()\
    .map(lambda data: data[1])\
    .collect()
  W = np.array(list(map(lambda x: np.array(x.flatten())[0], newW))).T
  w_broadcast.destroy()
  w_broadcast = sc.broadcast(W)
  newH = R_i.groupByKey()\
    .mapValues(lambda row:computeOptimizeMatrix(row,w_broadcast,lamb))\
    .sortByKey()\
    .map(lambda data: data[1])\
    .collect()
  H = np.array(list(map(lambda x: np.array(x.flatten())[0], newH))).T
  h_broadcast.destroy()
  h_broadcast = sc.broadcast(H)
  sse = M.map(lambda x: get_error_square(x[1][1], x[0], x[1][0])).reduce(lambda x,y: x+y)[0,0]
  count = M.count()
  mse = pow((sse/count), 0.5)
  print("Iteration %d:" % i)
  print("\nRMSE: %5.4f\n" % mse)


Iteration 0:

RMSE: 0.8346

Iteration 1:

RMSE: 0.7689

Iteration 2:

RMSE: 0.7434

Iteration 3:

RMSE: 0.7292

Iteration 4:

RMSE: 0.7204

Iteration 5:

RMSE: 0.7144

Iteration 6:

RMSE: 0.7101

Iteration 7:

RMSE: 0.7067

Iteration 8:

RMSE: 0.7040

Iteration 9:

RMSE: 0.7018

Iteration 10:

RMSE: 0.7000

Iteration 11:

RMSE: 0.6984

Iteration 12:

RMSE: 0.6969

Iteration 13:

RMSE: 0.6956

Iteration 14:

RMSE: 0.6945

Iteration 15:

RMSE: 0.6935

Iteration 16:

RMSE: 0.6926

Iteration 17:

RMSE: 0.6918

Iteration 18:

RMSE: 0.6910

Iteration 19:

RMSE: 0.6904

Iteration 20:

RMSE: 0.6898

Iteration 21:

RMSE: 0.6893

Iteration 22:

RMSE: 0.6888

Iteration 23:

RMSE: 0.6884

Iteration 24:

RMSE: 0.6880

Iteration 25:

RMSE: 0.6875

Iteration 26:

RMSE: 0.6872

Iteration 27:

RMSE: 0.6868

Iteration 28:

RMSE: 0.6865

Iteration 29:

RMSE: 0.6862

Iteration 30:

RMSE: 0.6859

Iteration 31:

RMSE: 0.6856

Iteration 32:

RMSE: 0.6853

Iteration 33:

RMSE: 0.6851

Iteration 34:

RMSE: 0.6

In [ ]:
np.array(newW[0].flatten())[0]

array([0.5297142 , 1.09126282, 0.69499938, 1.59845686, 0.65000493,
       0.99131188, 0.45204522, 0.19177449, 0.68120384, 0.35439614])

In [ ]:
np.array(list(map(lambda x: x.flatten(), newW))).T.shape

(10, 1, 943)

In [ ]:
M_count = len(sorted_items)
U_count = len(sorted_users)
F = numFactors
partitions = numWorkers
ITERATIONS = 2

In [ ]:
R = np.zeros((M_count, U_count))
rating_rdd = M.map(lambda x: (x[0], x[1][0], x[1][1])).collect()


In [ ]:
ms = np.matrix(np.random.rand(M_count, F))
us = np.matrix(np.random.rand(U_count, F))

Rb = sc.broadcast(R)
msb = sc.broadcast(ms)
usb = sc.broadcast(us)



for i in range(ITERATIONS):
    ms_ = sc.parallelize(range(M), partitions) \
        .map(lambda x: update(x, usb.value, Rb.value)) \
        .collect()
    # collect() returns a list, so array ends up being
    # a 3-d array, we take the first 2 dims for the matrix
    ms = np.matrix(np.array(ms_)[:, :, 0])
    msb = sc.broadcast(ms)

    us_ = sc.parallelize(range(U), partitions) \
        .map(lambda x: update(x, msb.value, Rb.value.T)) \
        .collect()
    us = np.matrix(np.array(us_)[:, :, 0])
    usb = sc.broadcast(us)

    error = rmse(R, ms, us)
    print("Iteration %d:" % i)
    print("\nRMSE: %5.4f\n" % error)


In [ ]:
def get_error_square(rating, i, j):
  pred = us[[i], :].dot(ms[[j], :].T)[0][0]
  return (rating - pred)**2

ITERATIONS = 100
for i in range(ITERATIONS):
    ms_ = sc.parallelize(range(M_count), partitions) \
        .map(lambda x: update(x, usb.value, Rb.value)) \
        .collect()
    # collect() returns a list, so array ends up being
    # a 3-d array, we take the first 2 dims for the matrix
    ms = np.matrix(np.array(ms_)[:, :, 0])
    msb = sc.broadcast(ms)

    us_ = sc.parallelize(range(U_count), partitions) \
        .map(lambda x: update(x, msb.value, Rb.value.T)) \
        .collect()
    us = np.matrix(np.array(us_)[:, :, 0])
    usb = sc.broadcast(us)

    
    sse = M.map(lambda x: get_error_square(x[1][1], sorted_users[x[0]], sorted_items[x[1][0]])).reduce(lambda x,y: x+y)[0, 0]
    count = M.count()
    rmse = pow(sse/count, 0.5)
    error = get_rmse(R, ms, us)
    print("Iteration %d:" % i)
    print("\nRMSE: %5.4f" % error)
    print("\nGlobal RMSE: %5.4f\n" % rmse)


In [ ]:
usb.value.shape[1]

In [ ]:
def get_error_square(rating, i, j):
  pred = us[:, [i]].T.dot(ms[:, [j]])[0][0]
  return (rating - pred)**2

In [ ]:
ms_ = sc.parallelize(range(M_count), partitions) \
    .map(lambda x: update(x, usb.value, Rb.value)) \
    .collect()
# collect() returns a list, so array ends up being
# a 3-d array, we take the first 2 dims for the matrix
ms = np.matrix(np.array(ms_)[:, :, 0])
msb = sc.broadcast(ms)

us_ = sc.parallelize(range(U_count), partitions) \
    .map(lambda x: update(x, msb.value, Rb.value.T)) \
    .collect()
us = np.matrix(np.array(us_)[:, :, 0])
usb = sc.broadcast(us)
mse = get_rmse(M, ms, us)
error = rmse(R, ms, us)
print("Iteration %d:" % i)
print("\nErr: %5.4f\n" % error)
print("\nRMSE: %5.4f\n" % rmse)


In [ ]:
R

In [ ]:
XtX = usb.value.T * usb.value
inv(XtX + 0.01 * np.eye(F)) * Xty

In [ ]:
Xty = usb.value.T * Rb.value[6, :].T
Xty